: 

In [ ]:
#################################################
#                   Imports                     #
#################################################

import sys

############## imports RetinaFace ###############
import tensorflow as tf

from retinaface import RetinaFace
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import cv2
sys.path.insert(0, '../RetinaFace/deepface')
from deepface import DeepFace
import pandas as pd
# import tensorflow as tf

############## imports Image2text  ##############
import torch
from lavis.models import load_model_and_preprocess

########### imports Stable diffusion ############
from diffusers import StableDiffusionPipeline

################ imports SimSwap ################
import os
import sys
# # caution: path[0] is reserved for script path (or '' in REPL)
#sys.path.insert(1, '../Deepfake/SimSwap')

# #import test_multi_video_swapsingle
from insightface_func.face_detect_crop_multi import Face_detect_crop
# # adding Folder_SimSwap to the system path
sys.path.insert(0, '../Deepfake/SimSwap')
from models.models import create_model
from options.test_options import TestOptions
import glob
from torchvision import transforms
import torch.nn.functional as F
from util.reverse2original import reverse2wholeimage
from moviepy.video.io.ImageSequenceClip import ImageSequenceClip
from parsing_model.model import BiSeNet
from util.norm import SpecificNorm


import nvidia_smi
import gc

#################################################
#              Globale variabele                #
############################################SS#####

aantal_gezichten_gedetecteerd = 0
herkende_gezichten_embedding = pd.DataFrame()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
multisepcific_dir = "../images/wissel/"
temp_results_dir='./temp_results'  
save_path = "./output/test.mp4" 
#torch.cuda.empty_cache()
torch.cuda.memory_summary(device=None, abbreviated=False)
os.environ['PYTORCH_CUDA_ALLOC_CONF']='expandable_segments:True, max_split_size_mb:512'

#################################################
#                   Code                        #
#################################################

def wetenOverGPU():
    nvidia_smi.nvmlInit()

    handle = nvidia_smi.nvmlDeviceGetHandleByIndex(0)
    # card id 0 hardcoded here, there is also a call to get all available card ids, so we could iterate

    info = nvidia_smi.nvmlDeviceGetMemoryInfo(handle)

    print("Total memory:", info.total)
    print("Free memory:", info.free)
    print("Used memory:", info.used)

    nvidia_smi.nvmlShutdown()

: 

In [ ]:
def simSwap(video, frames_met_gezichten, img):
    # img = cv2 rgb 512, 512, 3 
    # omzetten naar tensor met extra dim batch # len =4 b,h,w,c
    transformer_Arcface = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])

    opt = TestOptions().parse()

    print('./insightface_func/models')
    app = Face_detect_crop(name='antelope', root='./insightface_func/models')
    app.prepare(ctx_id= 0, det_thresh=0.0, det_size=(640,640),mode='None')

    torch.nn.Module.dump_patches = True
    model = create_model(opt)
    model.eval()

    vcap = cv2.VideoCapture(video)
    if not vcap.isOpened():
        print("Can't open camera!")
        exit()

    frame_count = int(vcap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = vcap.get(cv2.CAP_PROP_FPS)

    # while ret:
    for frame_index in range(100): 

        # read a frame
        ret, frame = vcap.read()
        #print(type(frame))
        # ret == True if frame read correctly
        if not ret:
            print("Can't read frame, exiting...")
            break
        #wetenOverGPU()
        # The specific person to be swapped(source)
        if frame_index in frames_met_gezichten:
            if aantal_gezichten_gedetecteerd == 1:
                single_face_video_swap(frame, frame_index, opt, app, model, transformer_Arcface)
            else:
                target_id_norm_list, source_specific_id_nonorm_list = get_images_for_multi_videoswap(app, model, transformer_Arcface)
                
                #wetenOverGPU()
                #print("--------video path----------")
                multi_video_swap(frame, frame_index, target_id_norm_list,source_specific_id_nonorm_list, opt.id_thres, model, app)
        else:
            if not os.path.exists(temp_results_dir):
                os.mkdir(temp_results_dir)
            frame = frame.astype(np.uint8)
            cv2.imwrite(os.path.join(temp_results_dir, 'frame_{:0>7d}.jpg'.format(frame_index)), frame)

    model.to("cpu")

    # everything done
    # - release capture
    # - release video writer
    # - destroy all windows
    print("Retinaface done")
    vcap.release()
    #output.release()
    cv2.destroyAllWindows()

    print("maak video")


    return fps

def single_face_video_swap(frame, frame_index, opt, app, model, transformer_Arcface):
    crop_size = 224
    transformer = transforms.Compose([
            transforms.ToTensor(),
            #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])

    transformer_Arcface = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])

    with torch.no_grad():
        img = cv2.imread("../images/wissel/DST_1.jpg")
        #print("????????????????????????????????????")
        img_a_align_crop, _ = app.get(img,crop_size)
        img_a_align_crop_pil = Image.fromarray(cv2.cvtColor(img_a_align_crop[0],cv2.COLOR_BGR2RGB)) 
        img_a = transformer_Arcface(img_a_align_crop_pil)
        img_id = img_a.view(-1, img_a.shape[0], img_a.shape[1], img_a.shape[2])

        # pic_b = opt.pic_b_path
        # img_b_whole = cv2.imread(pic_b)
        # img_b_align_crop, b_mat = app.get(img_b_whole,crop_size)
        # img_b_align_crop_pil = Image.fromarray(cv2.cvtColor(img_b_align_crop,cv2.COLOR_BGR2RGB)) 
        # img_b = transformer(img_b_align_crop_pil)
        # img_att = img_b.view(-1, img_b.shape[0], img_b.shape[1], img_b.shape[2])

        # convert numpy to tensor
        img_id = img_id.cuda()
        # img_att = img_att.cuda()

        #create latent id
        img_id_downsample = F.interpolate(img_id, size=(112,112))
        latend_id = model.netArc(img_id_downsample)
        latend_id = F.normalize(latend_id, p=2, dim=1)

        single_video_swap(frame, frame_index, latend_id, model, app, opt.output_path)

def single_video_swap(frame, frame_index, id_vetor, swap_model, detect_model, temp_results_dir = './temp_results'):

    
    crop_size = 224
    use_mask = False

    spNorm = SpecificNorm()
    #swap_model.cpu()
    #reverse2wholeimage.cpu()

    #wetenOverGPU()
    # while ret:
    detect_results = detect_model.get(frame,crop_size)

    if detect_results is not None:
        # print(frame_index)
        #wetenOverGPU()
        if not os.path.exists(temp_results_dir):
                os.mkdir(temp_results_dir)
        frame_align_crop_list = detect_results[0]
        frame_mat_list = detect_results[1]
        swap_result_list = []
        frame_align_crop_tenor_list = []
        #wetenOverGPU()

        for frame_align_crop in range(6000000000):

            # BGR TO RGB
            # frame_align_crop_RGB = frame_align_crop[...,::-1]

            frame_align_crop_tenor = _totensor(cv2.cvtColor(frame_align_crop_list[0],cv2.COLOR_BGR2RGB))[None,...].cuda()

            swap_result = swap_model(None, frame_align_crop_tenor, id_vetor, None, True)[0]
            cv2.imwrite(os.path.join(temp_results_dir, 'frame_{:0>7d}.jpg'.format(frame_index)), frame)
            swap_result_list.append(swap_result)
            frame_align_crop_tenor_list.append(frame_align_crop_tenor)
#
 
        wetenOverGPU()

        torch.cuda.empty_cache()
        #wetenOverGPU()

        reverse2wholeimage(frame_align_crop_tenor_list,swap_result_list, frame_mat_list, crop_size, frame, \
            os.path.join(temp_results_dir, 'frame_{:0>7d}.jpg'.format(frame_index)), norm = spNorm)

    else:
        if not os.path.exists(temp_results_dir):
            os.mkdir(temp_results_dir)
        frame = frame.astype(np.uint8)
        cv2.imwrite(os.path.join(temp_results_dir, 'frame_{:0>7d}.jpg'.format(frame_index)), frame)

def get_images_for_multi_videoswap(app, model, transformer_Arcface):
    
    crop_size = 244
    source_specific_id_nonorm_list = []
    print("-------test-------------")
    source_path = os.path.join(multisepcific_dir,'SRC_*')
    print(source_path)
    source_specific_images_path = sorted(glob.glob(source_path))
    print(source_specific_images_path)
    with torch.no_grad():
        for source_specific_image_path in source_specific_images_path:
            #print(source_specific_image_path)
            specific_person_whole = cv2.imread(source_specific_image_path)
            #print(specific_person_whole)
            specific_person_align_crop, _ = app.get(specific_person_whole,crop_size)
            specific_person_align_crop_pil = Image.fromarray(cv2.cvtColor(specific_person_align_crop[0],cv2.COLOR_BGR2RGB)) 
            specific_person = transformer_Arcface(specific_person_align_crop_pil)
            specific_person = specific_person.view(-1, specific_person.shape[0], specific_person.shape[1], specific_person.shape[2])
            # convert numpy to tensor
            specific_person = specific_person.cuda()
            #create latent id
            specific_person_downsample = F.interpolate(specific_person, size=(112,112))
            specific_person_id_nonorm = model.netArc(specific_person_downsample)
            source_specific_id_nonorm_list.append(specific_person_id_nonorm.clone())


        # The person who provides id information (list)
        target_id_norm_list = []
        target_path = os.path.join(multisepcific_dir,'DST_*')
        print(target_path)
        target_images_path = sorted(glob.glob(target_path))
        print(target_images_path)
        
        for target_image_path in target_images_path:
            img_a_whole = cv2.imread(target_image_path)
            print(target_image_path)
            img_a_align_crop, _ = app.get(img_a_whole,crop_size)
            img_a_align_crop_pil = Image.fromarray(cv2.cvtColor(img_a_align_crop[0],cv2.COLOR_BGR2RGB)) 
            img_a = transformer_Arcface(img_a_align_crop_pil)
            img_id = img_a.view(-1, img_a.shape[0], img_a.shape[1], img_a.shape[2])
            # convert numpy to tensor
            img_id = img_id.cuda()
            #create latent id
            img_id_downsample = F.interpolate(img_id, size=(112,112))
            latend_id = model.netArc(img_id_downsample)
            latend_id = F.normalize(latend_id, p=2, dim=1)
            target_id_norm_list.append(latend_id.clone())

        assert len(target_id_norm_list) == len(source_specific_id_nonorm_list), "The number of images in source and target directory must be same !!!"
    return source_specific_id_nonorm_list, target_id_norm_list

def multi_video_swap(frame, frame_index, target_id_norm_list,source_specific_id_nonorm_list, id_thres, swap_model, detect_model):
    wetenOverGPU()
    crop_size = 224
    no_simswaplogo = True
    logoclass = None

    spNorm = SpecificNorm()
    mse = torch.nn.MSELoss().cuda()

    # while ret:
    detect_results = detect_model.get(frame,crop_size)
    print("a")
    if detect_results is not None:
        # print(frame_index)
        if not os.path.exists(temp_results_dir):
                os.mkdir(temp_results_dir)
        frame_align_crop_list = detect_results[0]
        print(f"lengte van frame_align_crop_list: {len(frame_align_crop_list)}")
        frame_mat_list = detect_results[1]

        print("b")
        wetenOverGPU()
        id_compare_values = [] 
        frame_align_crop_tenor_list = []
        for frame_align_crop in frame_align_crop_list:

            # BGR TO RGB
            # frame_align_crop_RGB = frame_align_crop[...,::-1]

            frame_align_crop_tenor = _totensor(cv2.cvtColor(frame_align_crop,cv2.COLOR_BGR2RGB))[None,...].cuda()
            #print("hier")
            frame_align_crop_tenor_arcnorm = spNorm(frame_align_crop_tenor)
            #print("o")
            frame_align_crop_tenor_arcnorm_downsample = F.interpolate(frame_align_crop_tenor_arcnorm, size=(112,112))#.cpu()
            #print("l")
            frame_align_crop_crop_id_nonorm = swap_model.netArc(frame_align_crop_tenor_arcnorm_downsample)#.cpu()
            #print("huh")
            id_compare_values.append([])
            #print("nee")
            for source_specific_id_nonorm_tmp in source_specific_id_nonorm_list:
                id_compare_values[-1].append(mse(frame_align_crop_crop_id_nonorm,source_specific_id_nonorm_tmp).detach().cpu().numpy())
                #print("m")
            frame_align_crop_tenor_list.append(frame_align_crop_tenor)
            break
        torch.cuda.empty_cache()
        print("c")
        id_compare_values_array = np.array(id_compare_values).transpose(1,0)
        wetenOverGPU()
        try:
            min_indexs = np.argmin(id_compare_values_array,axis=0)
            #print(min_indexs)
        except ValueError:
            print(" ")
        min_value = np.min(id_compare_values_array,axis=0)
        print("d")
        #print(min_value)
        swap_result_list = [] 
        swap_result_matrix_list = []
        swap_result_ori_pic_list = []
        wetenOverGPU()
        for tmp_index, min_index in enumerate(min_indexs):
            #os.environ['PYTORCH_CUDA_ALLOC_CONF']='expandable_segments:True, max_split_size_mb:512'
            #os.environ["SUNO_USE_SMALL_MODELS"] = "True"
            #os.environ["SUNO_OFFLOAD_CPU"] = "True"
            if min_value[tmp_index] < id_thres:
                swap_result = swap_model(None, frame_align_crop_tenor_list[tmp_index], target_id_norm_list[min_index], None, True)[0]
                print("hier-")
                wetenOverGPU()
                swap_result_list.append(swap_result)
                swap_result_matrix_list.append(frame_mat_list[tmp_index])
                swap_result_ori_pic_list.append(frame_align_crop_tenor_list[tmp_index])
                print("m")
                wetenOverGPU()
            else:
                pass
            wetenOverGPU()
        #os.environ['PYTORCH_CUDA_ALLOC_CONF']='expandable_segments:True, max_split_size_mb:512'
        print("e")


        if len(swap_result_list) !=0:
            print("wissel gebeurt")
            reverse2wholeimage(swap_result_ori_pic_list,swap_result_list, swap_result_matrix_list, crop_size, frame, logoclass,\
                os.path.join(temp_results_dir, 'frame_{:0>7d}.jpg'.format(frame_index)), pasring_model = None, use_mask = False, norm = spNorm)
        else:
            if not os.path.exists(temp_results_dir):
                os.mkdir(temp_results_dir)
            frame = frame.astype(np.uint8)
            if not no_simswaplogo:
                frame = logoclass.apply_frames(frame)
            cv2.imwrite(os.path.join(temp_results_dir, 'frame_{:0>7d}.jpg'.format(frame_index)), frame)
        print("f")
    else:
        if not os.path.exists(temp_results_dir):
            os.mkdir(temp_results_dir)
        frame = frame.astype(np.uint8)
        cv2.imwrite(os.path.join(temp_results_dir, 'frame_{:0>7d}.jpg'.format(frame_index)), frame)
    print("e")
    return

def _totensor(array):
    tensor = torch.from_numpy(array)
    img = tensor.transpose(0, 1).transpose(0, 2).contiguous()
    return img.float().div(255)

: 

In [ ]:
aantal_gezichten_gedetecteerd = 1
#if not os.path.exists(multisepcific_dir):
#    os.mkdir(temp_results_dir)
#else:
#    os.system('rm -rf ' +multisepcific_dir + '*')
#print("test")
#wetenOverGPU() # moet ongeveer 8G vrij hebben
orginele_video = '/home/student/Documents/AnyDesk/masterproef/images/gezicht2.mp4'

#retinaface_frames_met_gezichten = retinaface(orginele_video)
#print(retinaface_frames_met_gezichten[:10])
#antwoorden = image2text()
#print(antwoorden)
#img = stableDiffussion(antwoorden) # returns an image (fake face)
# cv2.img in rgb shape = (512, 512, 3)
crop_size = 512

img = cv2.imread("../images/wissel/DST_1.jpg")
img_cropped = img[:crop_size,:crop_size]
retinaface_frames_met_gezichten = range(50)
fps = simSwap(orginele_video, retinaface_frames_met_gezichten, img_cropped) # afbeelding van stable diffusion
wetenOverGPU()
#print("RetinaFace gelukt")

# image_filename_list = []

path = os.path.join(temp_results_dir,'*.jpg')
image_filenames = sorted(glob.glob(path))
print(image_filenames)

clips = ImageSequenceClip(image_filenames,fps = fps)
clips.write_videofile(save_path, codec="libx264")
